# Classify NDWI image using defined thresholds

## Load packages

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../Scripts')
from deafrica_spatialtools import xr_rasterize

In [2]:
# define area name

area_name = 'Sahel'
if not os.path.exists(area_name): os.mkdir(area_name)

## Convert to one mosaic (only do it once)

In [3]:
# make tif

if not os.path.exists(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.vrt"):
    os.chdir('NDWI_composite')
    os.system(f"gdalbuildvrt {area_name.lower()}_NDWI_mosaic.vrt {area_name.lower()}_NDWI_tile*.tif")
    #os.system("gdal_translate "\
    #   "-co BIGTIFF=YES "\
    #   "-co COMPRESS=DEFLATE "\
    #   "-co ZLEVEL=9 "\
    #   "-co PREDICTOR=1 "\
    #   "-co TILED=YES "\
    #   "-co BLOCKXSIZE=1024 "\
    #   "-co BLOCKYSIZE=1024 "\
    #   +f"{area_name.lower()}_NDWI_mosaic.vrt "+ f"{area_name.lower()}_NDWI_mosaic.tif")
    os.chdir('../')

## Load NDWI mosaic

In [5]:
ds = xr.open_rasterio(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.vrt").squeeze()

In [7]:
dataset = ds.to_dataset(name='ndwi')

## Classify into bins of different water detection frequencies

In [11]:
# fix thresholds
thresh = [-0.07, -0.05, -0.03, 0.03]

n_class = len(thresh)+1
frac_sample = [0.1, 0.1, 0.2, 0.3, 0.3]

In [11]:
label = np.zeros_like(dataset.ndwi.values, dtype=np.uint8)

label +=(dataset.ndwi.values<thresh[0]).astype(np.uint8)*1
for i in range(2, n_class):
    label += ((dataset.ndwi.values>=thresh[i-2]) & (dataset.ndwi.values<thresh[i-1])).astype(np.uint8)*i

label += (dataset.ndwi.values>=thresh[-1]).astype(np.uint8)*n_class

dataset['label'] = ('y','x'), label

# ndwi=0 is not valid
dataset['label'] = dataset.label.where(dataset.ndwi!=0, 0)

dataset['label'].attrs = dataset.ndwi.attrs

/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater_equal
  
/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
/env/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater_equal
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# free up memory
del ds

## Clip to AEZ (excluding large_water_bodies)

In [ ]:
#load shapefile
#gdf = gpd.read_file(f'../../shapes/simplified_AEZs/{area_name}.shp')
gdf = gpd.read_file(f'../../shapes/AEZs_ExcludeLargeWB/AEZs_ExcludeLargeWB_update_{area_name}.shp')

#rasterize shapeile
mask = xr_rasterize(gdf=gdf,
                     da=dataset.label)

dataset['label'] = dataset.label.where(mask, 0)

In [12]:
# save classes

from datacube.utils.cog import write_cog

write_cog(dataset.label, f'{area_name}/{area_name}_label.tif')

PosixPath('Southern/Southern_label.tif')